In [ ]:
! pip install -r ../../requirements.txt
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [9]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('/home/sas/GitRepos/AlexaGPT2/ML/train.txt','/home/sas/GitRepos/AlexaGPT2/ML/test.txt',tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (59933 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./model_parameters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=20, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=16, # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=400, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [11]:
trainer.train()

Step,Training Loss
500,2.904600


TrainOutput(global_step=600, training_loss=2.7602442423502604, metrics={'train_runtime': 136.7766, 'train_samples_per_second': 4.387, 'total_flos': 894533071011840, 'epoch': 20.0})

In [12]:
trainer.save_model()

In [13]:
from transformers import pipeline

generator = pipeline('text-generation',model="./model_parameters", tokenizer='gpt2', config={'max_length':10000})


In [14]:
def generate_response(input):
    print(input)
    return generator(input, do_sample=True, max_length=len(input.split())+60, top_k=50, top_p=0.95, num_returned_sequences=1)[0]['generated_text']


In [17]:
print(generate_response(""))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




All eyes are upon us
And the stars to my left
Are filled with cries of joy and pain
All round me
As I approach you
There are many who claim
The throne of queens
Who claim
The throne of kings
Are all but strangers
From every corner
